<a href="https://colab.research.google.com/github/Licencinho/PMA/blob/main/run_PMA_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install flask-ngrok
!pip install pyngrok
!pip install flask-wtf

In [12]:
!ngrok authtoken "26BvbMOktqjiQ2Ibfb05r7jzTcL_4UJaagneSjXiB6qeUZXzT"

Authtoken saved to configuration file: C:\Users\alons/.ngrok2/ngrok.yml


In [5]:
#only execute in Google Colab. Not needed in local
!git clone "https://github.com/Licencinho/PMA"
import os
os.chdir("/content/PMA")

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '/content/PMA'

In [29]:
from flask import *
from flask_ngrok import run_with_ngrok
import pandas as pd
from flask_wtf import FlaskForm
from wtforms import *
from wtforms import validators
from wtforms.validators import *
import datetime


app = Flask(__name__)
app.config['SECRET_KEY'] = 'super-secret-impossible-impopossible' # this should go in a .env file

from flask_wtf.csrf import CSRFProtect
csrf = CSRFProtect(app)

app_data = {
    "name":         "Plannerd",
    "description":  "A basic Flask app for students to plan their dissertations",
    "author":       "Alonso Olate, adaptation from Peter Simeth's flask template",
    "html_title":   "Plannerd",
    "project_name": "Plannerd",
    "keywords":     "flask, webapp, dissertation, planning, python, plannerd"
}

class WorkloadForm(FlaskForm):
	module_number = IntegerField('How many modules does your Master have? (Required)', validators=[NumberRange(min=5, max=15, message='Please put a number between 5 and 15')], default=0)
	hours_in_dissertation = IntegerField('How many hours have you worked in your dissertation so far?', validators=[NumberRange(min=0, max=900, message='Please put a number between 0 and 900')], default=0)
	module_1_boolean = BooleanField('Module 1')
	module_2_boolean = BooleanField('Module 2')
	module_3_boolean = BooleanField('Module 3')
	module_4_boolean = BooleanField('Module 4')
	module_5_boolean = BooleanField('Module 5')
	module_6_boolean = BooleanField('Module 6')
	module_7_boolean = BooleanField('Module 7')
	module_8_boolean = BooleanField('Module 8')
	module_9_boolean = BooleanField('Module 9')
	module_10_boolean = BooleanField('Module 10')
	module_1_PMA_percentage = IntegerField('Module 1', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_2_PMA_percentage = IntegerField('Module 2', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_3_PMA_percentage = IntegerField('Module 3', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_4_PMA_percentage = IntegerField('Module 4', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_5_PMA_percentage = IntegerField('Module 5', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_6_PMA_percentage = IntegerField('Module 6', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_7_PMA_percentage = IntegerField('Module 7', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_8_PMA_percentage = IntegerField('Module 8', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_9_PMA_percentage = IntegerField('Module 9', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)
	module_10_PMA_percentage = IntegerField('Module 10', validators=[NumberRange(min=0, max=100, message='Please put a number between 0 and 100')], default=0)

class LoginForm(FlaskForm):
	email = EmailField('Email', validators=[DataRequired()])
	password = PasswordField('Password', validators =[DataRequired()])


@app.route('/')
def index():
	login_message = ""
	form = LoginForm()
	df = pd.read_csv("users.csv")
	return render_template('login.html', df=df, form = form, login_message = login_message, app_data=app_data)

@app.route('/login', methods=('GET', 'POST'))
def login():
	login_message = ""
	form = LoginForm()
	df = pd.read_csv("users.csv")
	if request.method == "POST":
		if form.validate_on_submit():

			# Search in Pandas adapted from https://www.interviewqs.com/ddi-code-snippets/rows-cols-python.
			
			dfFound = df.loc[(df['email'] == request.form.get("email")) & (df['password'] == request.form.get("password"))]
			
			#If dfFound has one row or more, it means we found a user with matching email and password
			count_row = dfFound.shape[0]
			if count_row > 0:
				return render_template('index.html', app_data=app_data)

	login_message = "Your credentials are not correct. Please try again"
	return render_template('login.html', df=df, form=form, login_message = login_message, app_data=app_data)


@app.route('/calendar')
def calendar():
    return render_template('calendar.html', app_data=app_data)


@app.route('/workload')
def workload():
    form = WorkloadForm()
    return render_template('workload.html', form = form, app_data=app_data)

@app.route('/results')
def results():
    formResults = {}
    return render_template('results.html', formResults = formResults, app_data=app_data)


@app.route('/submit', methods=('GET', 'POST'))
def submit():
	form = WorkloadForm()
	formResults = {}
	if request.method == "POST":
		if form.validate_on_submit():
			formResults['hours_in_dissertation'] = float(request.form.get("hours_in_dissertation"))

			# Calculate how many hours spent in class in modules
			formResults['hours_in_modules'] = 0
			if(request.form.get("module_1_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40
			if(request.form.get("module_2_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40
			if(request.form.get("module_3_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40	
			if(request.form.get("module_4_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40
			if(request.form.get("module_5_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40
			if(request.form.get("module_6_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40
			if(request.form.get("module_7_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40
			if(request.form.get("module_8_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40
			if(request.form.get("module_9_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40
			if(request.form.get("module_10_boolean")):
				formResults['hours_in_modules'] = float(formResults['hours_in_modules']) + 40

			# Calculate how many hours spent in PMA work
			formResults['hours_in_PMAs'] = round((float(request.form.get("module_1_PMA_percentage")) + float(request.form.get("module_2_PMA_percentage")) + float(request.form.get("module_3_PMA_percentage")) + float(request.form.get("module_4_PMA_percentage")) + float(request.form.get("module_5_PMA_percentage")) + float(request.form.get("module_6_PMA_percentage")) + float(request.form.get("module_7_PMA_percentage")) + float(request.form.get("module_8_PMA_percentage")) + float(request.form.get("module_9_PMA_percentage")) + float(request.form.get("module_10_PMA_percentage"))) * 0.6, 0)

			# Calculate how many hours spent in modules and PMA work
			formResults['hours_in_modules_and_PMAs'] = round(float(formResults['hours_in_modules']) + float(formResults['hours_in_PMAs']), 0)

			# Calculate how many hours to do in modules
			formResults['module_hours_to_do'] = round(float(request.form.get("module_number"))*100 - float(formResults ['hours_in_modules_and_PMAs']), 0)

			# Calculate how many hours to do in dissertation
			formResults['dissertation_hours_to_do'] =  round(900 - float(request.form.get("hours_in_dissertation")), 0)

			# Calculate total work hours to do
			formResults['total_hours_to_do'] =  round(float(formResults['module_hours_to_do']) + float(formResults['dissertation_hours_to_do']), 0)

			# Calculate weeks remaining to submission
			submissionDate = datetime.datetime(2022,9,6)
			today = datetime.datetime.now()
			difference = submissionDate - today
			formResults['weeks_to_submission'] = round(difference.days/7, 1)

			# Calculate hours per week to work from now
			formResults['hours_per_week'] = round(float(formResults['total_hours_to_do'])/float(formResults['weeks_to_submission']), 1)

			# Calculate hours per day in a 7 day week
			formResults['hours_per_day_in_7_day_week'] = round(float(formResults['hours_per_week'])/7, 1)
			# Calculate hours per day in a 6 day week
			formResults['hours_per_day_in_6_day_week'] = round(float(formResults['hours_per_week'])/6, 1)
			# Calculate hours per day in a 5 day week
			formResults['hours_per_day_in_5_day_week'] = round(float(formResults['hours_per_week'])/5, 1)

			return render_template('results.html', formResults=formResults, app_data=app_data)

	return render_template('workload.html', form=form, app_data=app_data)
	

run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Mar/2022 14:05:06] "POST /login HTTP/1.1" 200 -


 * Running on http://719f-31-205-0-51.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Mar/2022 14:05:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:05:21] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:05:32] "GET /workload HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:05:38] "GET /calendar HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:05:41] "GET /calendar HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:05:41] "GET /workload HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:05:54] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:05:55] "GET /submit HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:06:03] "GET /workload HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:06:04] "GET /calendar HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:06:07] "GET /workload HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 14:06:08] "GET / HTTP/1.1" 200 -
